# Homework 3 - Algorithmic Methods of Data Mining

In [1]:
#Package import
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import os
import time
import re
import nltk
import csv

# 1. Data Collection

#### We start from the list of animes to include in your corpus of documents. In particular, we focus on the top animes ever list. From this list we want to collect the url associated to each anime in the list. The list is long and splitted in many pages. We ask you to retrieve only the urls of the animes listed in the first 400 pages (each page has 50 animes so you will end up with 20000 unique anime urls).

#### The output of this step is a .txt file whose single line corresponds to an anime's url.

#### 1.1. Get the list of animes

In [2]:
def extract_urls_f(num_of_pages):
    #Initialize page zero
    num_pages = 0
    #Create a list to contain all extracted information
    links_of_inter = []

    while(num_pages<num_of_pages):
        #Connecting to the webpage
        URL = f"https://myanimelist.net/topanime.php?limit={num_pages}"
        page = requests.get(URL)
    
        #Parsing the page
        soup = BeautifulSoup(page.content, "html.parser")
    
        #Extract all within a certain class
        tag_of_inter = soup.find_all('a',{'class':'hoverinfo_trigger fl-l ml12 mr8'})
    
        #Extract URL of each anime
        for e in tag_of_inter:
            links_of_inter.append(e.get('href'))
    
        #Increasing number of pages
        num_pages += 50
    
    return links_of_inter

In [3]:
def create_txt_f(list_to_convert):
    #Creating a txt based from the list of URLs
    with open("url_of_interest.txt", "w") as output:
        output.write('\n'.join(str(line) for line in list_to_convert))
    
    #Write all list in a single line
    #with open("url_of_interest.txt", "w") as output:
    #    output.write(str(links_of_inter))

In [ ]:
# Extrac URLs
extracted_urls = extract_urls_f(20000)

In [ ]:
#Create the txt file
create_txt_f(extracted_urls)

In [ ]:
len(extracted_urls)

##### Read the txt file which we should already have it the cwd

In [3]:
#Read the txt file
with open('url_of_interest.txt', encoding = 'utf8') as f:
    extracted_urls = f.readlines()

In [4]:
len(extracted_urls)

19131

#### 1.2. Crawl animes

Once you get all the urls in the first 400 pages of the list, you:

Download the html corresponding to each of the collected urls.
After you collect a single page, immediately save its html in a file. In this way, if your program stops, for any reason, you will not lose the data collected up to the stopping point. More details in Important (2).
Organize the entire set of downloaded html pages into folders. Each folder will contain the htmls of the animes in page 1, page 2, ... of the list of animes.

In [ ]:
principal_wd = os.getcwd()
t = 0

starting_point = 3258 #This can be changed to select from which URL we will start extracting info
ending_point = 6500 #Take care and modify

for i in range(starting_point,ending_point):
    if(i%50==0):
        #*******FOLDER CREATION************#
        page_identifier = i-(49*t)
        #Creating a new folder
        # Leaf directory
        directory = f"page_{page_identifier}.html"
    
        # Parent Directories
        parent_dir = principal_wd
        
        # Path
        path = os.path.join(parent_dir, directory)
    
        # Create the directory
        os.makedirs(path)
        print("Directory '%s' created" %directory)
        
        #Change the directory! 
        os.chdir(path)
        
        #Increase t by 1
        t += 1
        #*******FOLDER CREATION************#
        
        
    time.sleep(5)    
    #*******HTML EXTRACTION************#
    #Take care a lot of info here!!
    #Connecting to the webpage
    URL = extracted_urls[i]
    page = requests.get(URL)
    
    #Parsing the page
    soup_data = BeautifulSoup(page.content, "html.parser")
    #*******HTML EXTRACTION************#
    
    #*******SAVE HTML FILE************#
    with open(f"article_{i}.html", "w", encoding = 'utf8') as file:
        file.write(str(soup_data))
    print(f"Article {i} successfully written!")

#### 1.3 Parse downloaded pages

At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

* Anime Name (to save as `animeTitle`): String
* Anime Type (to save as `animeType`): String
* Number of episode (to save as `animeNumEpisode`): Integer
* Release and End Dates of anime (to save as `releaseDate` and `endDate`): Convert both release and end date into datetime format.
* Number of members (to save as `animeNumMembers`): Integer
* Score (to save as `animeScore`): Float
* Users (to save as `animeUsers`): Integer
* Rank (to save as `animeRank`): Integer
* Popularity (to save as `animePopularity`): Integer
* Synopsis (to save as `animeDescription`): String
* Related Anime (to save as `animeRelated`): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.
* Characters (to save as `animeCharacters`): List of strings.
* Voices (to save as `animeVoices`): List of strings
* Staff (to save as `animeStaff`): Include the staff name and their responsibility/task in a list of lists.

In [5]:
def making_several_appends(list_to_append, *element):
    list_to_append.extend(element)

In [6]:
def coincidence_by_label(soup, label):
    return soup.find("span", text=re.compile(label)).next_sibling

In [7]:
def anime_Title_f(bs,list_name): #change

    #l = [None] * 1 #Define the list with length 1
    
    #Extract all within a certain class
    tag_of_inter = bs.find('h1',{'class':'title-name h1_bold_none'})

    a = tag_of_inter.text
    list_name.append(a) #Change
    return a

In [8]:
def animeType_f(bs,list_name): 
    list_name.append(bs.find(text="Type:").findNext('a').contents[0]) 
    return(bs.find(text="Type:").findNext('a').contents[0])

In [9]:
def animeNumEpisode_f(bs,list_name):
    list_name.append(int(coincidence_by_label(bs, "Episodes").strip()))
    return(coincidence_by_label(bs, "Episodes").strip())

In [10]:
def release_date_f(bs,list_name):
    dates = coincidence_by_label(bs, "Aired").strip()
    release = re.findall(r".+(?=to)", str(dates))
    end = re.findall(r"(?<=to).+", str(dates))
    list_name.append(release)
    list_name.append(end)
    return release,end

In [46]:
def animeNumMembers_f(bs,list_name):
    tag_of_inter = bs.find_all('span',{'class':'numbers members'})
    p = (re.findall(r"(?<=<strong>).+(?=</strong>)", str(tag_of_inter))[0])
    if p == 'Unknown':
        p = ''
    else:
        p = int(p.replace(',', ''))
    list_name.append(p)
    return p

In [12]:
def animeScore_f(bs, list_name):
    score = float(bs.find('div', class_ = 'fl-l score').find('div').text)
    list_name.append(score)
    return score

In [13]:
def animeUsers_f(bs, list_name):
    tag_of_inter = bs.find('div', {'class':'fl-l score'})
    users = int(''.join(re.findall(r'\d+', tag_of_inter.get('data-user'))))
    list_name.append(users)
    return users

In [14]:
def animeRank_f(bs, list_name):
    tag_of_inter = bs.find_all('span',{'class':'numbers ranked'})
    rank = int(re.findall(r"(?<=<strong>#).+(?=</strong>)", str(tag_of_inter))[0])
    list_name.append(rank)
    return rank

In [15]:
def animePopularity_f(bs, list_name):
    tag_of_inter = bs.find_all('span',{'class':'numbers popularity'})
    pop = int(re.findall(r"(?<=<strong>#).+(?=</strong>)", str(tag_of_inter))[0])
    list_name.append(pop)
    return pop    

In [16]:
def animeDescription_f(bs, list_name):
    synopsis = bs.find('p', itemprop='description').text
    list_name.append(synopsis)
    return synopsis

In [47]:
### THIS IS A TEST CELL ###
# Remember to remove it #
os.chdir("C:/Users/dcacc/Desktop/File/DAVIDE/Università ROMA/Algorithmic Methods of Data Mining/Html Folders/page_0.html")
html = open("article_0.html", 'r', encoding = 'utf8').read()
bs = BeautifulSoup(html, 'html.parser')
anime_info = []
animeNumMembers_f(bs, anime_info)
### THIS IS A TEST CELL ###

2675906

In [48]:
folder_to_begin = 0 #MUST CUSTOMIZE
folder_to_end = 3 #MUST CUSTOMIZE

for folder_identifier in range(folder_to_begin, folder_to_end):
    #*******THIS MUST BE CUSTOMIZED******
    #os.chdir(f"C:/Users/dcacc/Desktop/File/DAVIDE/Università ROMA/Algorithmic Methods of Data Mining/Html Folders/page_{folder_identifier}.html")
    os.chdir(f"/Users/brauliovillalobos/Documents/Data Science Master's Degree - Sapienza 2021 2023/ADM - Algorithms of Data Mining/HW_3/Pages_to_extract/page_{folder_identifier}.html")
    #*******THIS MUST BE CUSTOMIZED******
    
    if folder_identifier == 0:
        for anime_identifier in range(50):
            #Open the html that is going to be parsed
            html = open(f"article_{anime_identifier}.html", 'r', encoding = 'utf8').read()
    
            #Creating a list to consolidate
            anime_info = []
    
            #Parsing the HTML using BS
            bs = BeautifulSoup(html) 
    
            #Start calling functions
            animeTitle = anime_Title_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeType = animeType_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeNumEpisode = animeNumEpisode_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            releaseDate, endDate = release_date_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeNumMembers = animeNumMembers_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeScore = animeScore_f(bs, anime_info) #Keeping variable assignation for debugging purpose
            animeUsers = animeUsers_f(bs, anime_info)
            animeRank = animeRank_f(bs, anime_info)
            animePopularity = animePopularity_f(bs, anime_info)
            animeDescription = animeDescription_f(bs, anime_info)
      
            #Write a csv
            with open(f'anime_{anime_identifier}.tsv', 'w', newline='', encoding = 'utf8') as f_output:
                tsv_output = csv.writer(f_output, delimiter='\t')
                tsv_output.writerow(anime_info)
    
    else:
        for anime_identifier in range((folder_identifier)*50, (folder_identifier+1)*50):
            #Open the html that is going to be parsed
            html = open(f"article_{anime_identifier}.html", 'r', encoding = 'utf8').read()
    
            #Creating a list to consolidate
            anime_info = []
    
            #Parsing the HTML using BS
            bs = BeautifulSoup(html) 
    
            #Start calling functions
            animeTitle = anime_Title_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeType = animeType_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeNumEpisode = animeNumEpisode_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            releaseDate,endDate = release_date_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeNumMembers = animeNumMembers_f(bs, anime_info) #Keeping variable assignation for debugging purposes
            animeScore = animeScore_f(bs, anime_info) #Keeping variable assignation for debugging purpose
            animeUsers = animeUsers_f(bs, anime_info)
            animeRank = animeRank_f(bs, anime_info)
            animePopularity = animePopularity_f(bs, anime_info)
            animeDescription = animeDescription_f(bs, anime_info)
      
            #Write a csv
            with open(f'anime_{anime_identifier}.tsv', 'w', newline='', encoding = 'utf8') as f_output:
                tsv_output = csv.writer(f_output, delimiter='\t')
                tsv_output.writerow(anime_info)

ValueError: invalid literal for int() with base 10: 'Unknown'

# 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

Removing stopwords
Removing punctuation
Stemming
Anything else you think it's needed
For this purpose, you can use the nltk library.

In [ ]:
#If need to use nltk for the first time. 
#import nltk
#nltk.download()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def info_pre_processing_f(text_to_preprocess):
    #Import all english stopwords
    english_stopwords = stopwords.words('english')
    #Removing punctuation
    text_to_preprocess.translate(str.maketrans('', '', string.punctuation))
    # lowercase the text
    text_to_preprocess = text_to_preprocess.lower()
    # remove stopwords
    preprocessed_text = ""
    for word in preprocessed_text.split():
        if word not in english_stopwords:
            preprocessed_text += word + " " 
    return preprocessed_text

## 2.1 Conjunctive query

For the first version of the search engine, we narrow our interest on the Synopsis of each anime. It means that you will evaluate queries only with respect to the anime's description.

### 2.1.1 Create your index!

Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary of this format:

where document_i is the id of a document that contains the word.

Hint: Since you do not want to compute the inverted index every time you use the Search Engine, it is worth to think to store it in a separate file and load it in memory when needed.

### 2.1.2 Execute the query

Given a query, that you let the user enter:

What documents do we want?

Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

animeTitle
animeDescription
Url